In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from avalanche.benchmarks import SplitMNIST
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, forgetting_metrics
from avalanche.training import Naive
from avalanche.training.plugins import SynapticIntelligencePlugin

# --------------------
# 1. 网络定义
# --------------------
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x


# --------------------
# 2. 基准 (Benchmark) 构建
# --------------------
# 使用 SplitMNIST: 5 个小任务, 每个任务包含 2 个类别 (0-1, 2-3, ...)
# 使用 SplitMNIST: 5 个小任务，随机类别顺序
benchmark = SplitMNIST(
    n_experiences=5,
    return_task_id=False
)

# --------------------
# 3. 模型、优化器、损失和插件
# --------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN(num_classes=10).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Synaptic Intelligence 插件
si_plugin = SynapticIntelligencePlugin(
    si_lambda=0.1,   # 正则化强度, 可根据实验调整
    eps=0.1          # 防止除零
)

# --------------------
# 4. 定义训练策略
# --------------------
strategy = Naive(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    train_mb_size=32,
    train_epochs=3,
    eval_mb_size=64,
    device=device,
    plugins=[si_plugin]
)

# 日志记录
interactive_logger = InteractiveLogger()
# text_logger = TextLogger(open('si_splitmnist_logs.txt', 'w'))
# tensorboard_logger = TensorboardLogger('tb_logs')
strategy.evaluator.loggers = [interactive_logger]

# --------------------
# 5. 训练与评估
# --------------------
task_accuracies = []
print("Starting continual training on SplitMNIST with SI...")
for experience in benchmark.train_stream:
    print(f"\n--- Training on experience {experience.current_experience} ---")
    strategy.train(experience)
    print("Evaluation:")
    results = strategy.eval(benchmark.test_stream)
    task_accuracies.append(results)

print("Training completed!")


2025-07-07 11:47:00.412486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-07 11:47:00.450981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-07 11:47:00.462760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-07 11:47:00.487638: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-07 11:47:03.526536: W tensorflow/comp

Starting continual training on SplitMNIST with SI...

--- Training on experience 0 ---
-- >> Start of training phase << --
100%|██████████| 396/396 [00:07<00:00, 50.82it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0450
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9875
100%|██████████| 396/396 [00:07<00:00, 52.39it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0016
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9994
100%|██████████| 396/396 [00:07<00:00, 56.14it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0009
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9998
-- >> End of training phase << --
Evaluation:
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 34/34 [00:00<00:00, 94.10it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0021
	Top1_Acc_Exp/eval_phase/test_stream/Task000/

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x77443e3dc460>>
Traceback (most recent call last):
  File "/home/yangz2/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


100%|██████████| 369/369 [00:07<00:00, 49.85it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2981
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9123
 99%|█████████▉| 366/369 [00:07<00:00, 54.24it/s]

In [ ]:
import os
# Define the file path
file_path = "/home/yangz2/code/quantum_cl/results/list/splitminist_SI.pkl"

# Create directories if they don't exist
os.makedirs(os.path.dirname(file_path), exist_ok=True)  # <-- Add this line   

In [ ]:
import pickle
# 存储到文件
with open(file_path, "wb") as f:
    pickle.dump([task_accuracies], f)  